In [1]:
import re
import os
import json
from click import style

import numpy as np
import pandas as pd

from dash import Input, Output, html, dcc
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
from natsort import natsorted, index_natsorted , order_by_index
import dash
from dash import dcc, html, Output, Input, State
import dash_mantine_components as dmc
from dash_iconify import DashIconify
import dash_bootstrap_components  as dbc
from dash import  Input, Output, html
from dash.exceptions import PreventUpdate
import os
from uuid import uuid4
import flask
from dash import Dash,html
from werkzeug.middleware.dispatcher import DispatcherMiddleware
import flask
from werkzeug.serving import run_simple

import pandas as pd
import re
from helper.dtc_helper import *
from helper.parallel_plots import *
from helper.layout_setup.visualization_layout import *

In [2]:
carmodel =  pd.read_excel(r'data\car_models.xlsx', header = 0 , na_filter = False, engine='openpyxl', convert_float= False)
dtc_description  = pd.read_excel(r'data\dtc_info.xlsx',sheet_name= 'DTC', header=1, na_filter = False )

df =  pd.read_csv(r'data\dtc_data.csv', sep = ';', low_memory=False)
part_numbers = df['diagnostic_part_number'].unique()

did_info =  pd.read_excel(r'data\32360447 AC.xlsx',sheet_name= 'DID', header = 1 , na_filter = False, engine='openpyxl')
dtc_list = dropdown_list_for_filtering(df)
did_information = get_did_information(did_info)

C:\Users\ASRIDHAR\AppData\Local\Temp\ipykernel_12648\685978028.py:1: FutureWarning: convert_float is deprecated and will be removed in a future version.
  carmodel =  pd.read_excel(r'data\car_models.xlsx', header = 0 , na_filter = False, engine='openpyxl', convert_float= False)
C:\Users\ASRIDHAR\AppData\Local\Temp\ipykernel_12648\685978028.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  dtc_description  = pd.read_excel(r'data\dtc_info.xlsx',sheet_name= 'DTC', header=1, na_filter = False )
c:\Users\ASRIDHAR\OneDrive - Volvo Cars\Documents\Projects\Test\app(ARTLVC-11607)\helper\dtc_helper.py:79: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  did_info_index = did_information_modified[~did_information_modified['Compare Value'].str.contains(r'(=

In [3]:
app = JupyterDash(__name__,  external_stylesheets= [dbc.themes.BOOTSTRAP])

app.layout = html.Div([

    html.H1(
        children='Data Visualization and filter',
        style={
            'textAlign': 'center'
        }
    ),

    html.Br(),

    html.Div(children = [
        html.Label('Choose a model'),
        dmc.Space(h=10),
        dmc.Chips(
            id="model-type",
            data=[ 
                {"value":i , "label":i} for i in carmodel[carmodel['Part Number'].isin(part_numbers)]['Model'].to_list()
            ],
            value= "521H PP2",
        ),
        
    ]),

    dcc.Graph(id = 'dtc-plot'),

    html.Div([
        dbc.Row([
                dbc.Col(
                    [
                    
                    dbc.Row([ 
                        dbc.Col(create_graph_cards(1), width={"size": 5 }, style={'margin-right': '0px', 'margin-left': '0px'}),
                        dbc.Col(create_filter_cards(1), width={"size": 7}, style={'margin-right': '0px', 'margin-left': '0px'}),
                        ]),
                    html.Br(),
                    dbc.Row([
                        dbc.Col(dcc.Graph(id = 'snapshot-plot-1',className="h-150") , width = {"size": 5}, style={'margin-right': '0px', 'margin-left': '0px'}),
                        dbc.Col(dcc.Graph(id = 'snapshot-30-plot-1', className="h-100") , width = {"size": 7}, style={'margin-right': '0px', 'margin-left': '0px'})
  
                    ]),
                dcc.Store(id = "graph_plot_data_ex_1", storage_type ="local" ),

                ], width={ "offset": 0}, style = {"heights":350})
               
            ]),

    ])


])

@app.callback(
    Output('dtc-plot','figure'),
    Input('model-type', 'value'),
)
def update_graph(selected_model):
  
    part_number = carmodel[carmodel['Model'] == selected_model]['Part Number'].values[0]
    filter_df = df[df['diagnostic_part_number'].isin([part_number])]

    plot_dtc_count_df  = count_dtc_in_df(filter_df).sort_values(by='Count', ascending = False)

    fig = plot_count_histogram(plot_dtc_count_df)


    return fig

@app.callback(
    Output('select-did-1', 'data'),
    Input('select-snapshot-1','value'),
    Input('dtc-plot','selectedData')
)
def update_2040_did_dp_list(snapshot_value, selectedData):
    if snapshot_value is None or selectedData is None:
        raise PreventUpdate
    else:
        dtc_value = selectedData["points"][0]["customdata"][0]
        return [{'label': i, 'value': i} for i in list(dtc_list[dtc_value][snapshot_value].keys())]


@app.callback(
    Output('select-signal-1', 'data'),
    Input('select-did-1','value'),
    Input('select-snapshot-1','value'),
    Input('dtc-plot','selectedData')
)
def update_2040_signal_dp_list(did_value, snapshot_value, selectedData):
    if snapshot_value is None or did_value is None:
        raise PreventUpdate
    else:
        dtc_value = selectedData["points"][0]["customdata"][0]
        return [{'label': i, 'value': i} for i in dtc_list[dtc_value][snapshot_value][did_value]]

        
@app.callback(
    Output('snapshot-plot-1', 'figure'),
    [Input('select-snapshot-1', 'value'),
    Input('select-did-1',  'value'),
    Input('select-signal-1',  'value'),
    Input('dtc-plot', 'selectedData')]
)
def update_2040_snapshot_graph(snapshot_value, did_value, signal, selectData):
    try:
        if snapshot_value is None or signal is None:
            raise PreventUpdate
        else:
            dtc_value = selectData["points"][0]["customdata"][0]
           
        
            dtc_filtered_df = df[df.columns[df.columns.str.contains(dtc_value)==True]]
            dtc_filtered_df =  dtc_filtered_df.dropna(thresh = dtc_filtered_df.shape[0]*0.7,how='all',axis=0)
            dtc_filtered_df =  dtc_filtered_df.dropna(thresh = dtc_filtered_df.shape[0]*0.7,how='all',axis=1)
            plot_snapshot_df = dtc_filtered_df[dtc_filtered_df.columns[(dtc_filtered_df.columns.str.contains(rf'\d+/{dtc_value}/{snapshot_value}/{did_value}/{signal}'))]]
            trace_data = plot_snapshots_2040(plot_snapshot_df,did_information,did_value, signal)
            fig = go.Figure(trace_data) 
            fig.update_layout( 
                        height = 240,  
                    )
            return fig
            
    except IndexError:
        raise PreventUpdate


@app.callback(
    Output('x-values-graph-1','data'),
    Output('y-values-graph-1','data'),
    Output('customdata-graph-1','data'),
    Input('snapshot-plot-1', 'selectedData'),  
)
def update_select_data(selectData):
    selected_x_values = []
    selected_y_values = []
    selected_index = []
    if selectData is None:
        raise PreventUpdate
    elif len(selectData["points"]) > 1:
        for i in range(len(selectData["points"])):
            selected_x_values.append(selectData["points"][i]["x"])
            selected_y_values.append(selectData["points"][i]["y"])
            selected_index.extend(selectData["points"][i]["customdata"])
    else:
        selected_x_values = [selectData["points"][0]["x"]]
        selected_y_values = [selectData["points"][0]["y"]]
        selected_index = [selectData["points"][0]["customdata"]]
    return selected_x_values,selected_y_values,selected_index


@app.callback(
    Output('select-30-did-1', 'data'),
    Input('dtc-plot','selectedData')
)
def update_30_did_dp_list(selectData):
    if selectData is None:
        raise PreventUpdate
    else:
        dtc_value = selectData["points"][0]["customdata"][0]
        return [{'label': i, 'value': i} for i in list(dtc_list[dtc_value]['30'].keys())]

@app.callback(
    Output('select-30-signal-1', 'data'),
    Input('select-30-did-1','value'),
    State('dtc-plot','selectedData')
)
def update_30_signal_dp_list(did_value, selectData):
    if selectData is None or did_value is None:
        raise PreventUpdate
    else:
        dtc_value = selectData["points"][0]["customdata"][0]
        return [{'label': i, 'value': i} for i in dtc_list[dtc_value]['30'][did_value]]


@app.callback(
    Output('snapshot-30-plot-1','figure'),
    [Input('select-30-did-1', 'value'),
    Input('select-30-signal-1',  'value'),
    Input('dtc-plot', 'selectedData'),
    Input('customdata-graph-1','data'),]
)
def update_snapshot_graph( did_value, signal, dtc_hoverData, snapshot_hoverData):
    try:
        if signal is None  or snapshot_hoverData is None:
            raise PreventUpdate
        else:
            dtc_value = dtc_hoverData["points"][0]["customdata"][0]
            #index_data = snapshot_hoverData
        
            dtc_filtered_df = df[df.columns[df.columns.str.contains(dtc_value)==True]]
            dtc_filtered_df =  dtc_filtered_df.dropna(thresh = dtc_filtered_df.shape[0]*0.7,how='all',axis=0)
            dtc_filtered_df =  dtc_filtered_df.dropna(thresh = dtc_filtered_df.shape[0]*0.7,how='all',axis=1)

            trace_data, carindex = plot_snapshots_30(dtc_filtered_df,did_information, dtc_value,did_value, signal,snapshot_hoverData)
            fig = go.Figure(trace_data)

            return fig
            
    except IndexError:
        raise PreventUpdate

app.run_server(mode='inline', debug=True ) 

In [28]:
def create_snapshot_graph_layout_1(number):
    snapshot_plot_id = 'snapshot-plot-'+str(number)
    snapshot_30_plot_id = 'snapshot-30-plot-'+str(number)
    display = 'fyi-'+str(number)

    both_filter_graph_layout = html.Div([
            dbc.Row([
                dbc.Col(
                    [
                        dbc.Row([ 
                            dbc.Col(create_graph_cards(number), width={"size": 5 }, style={'margin-right': '0px', 'margin-left': '0px'}),
                            dbc.Col(create_filter_cards(number), width={"size": 7}, style={'margin-right': '0px', 'margin-left': '0px'}),
                            ]),
                        html.Br(),
                        dbc.Row([
                            dbc.Col(dcc.Graph(id = snapshot_plot_id,className="h-100") , width = {"size": 5}, style={'margin-right': '0px', 'margin-left': '0px','height': 240}),
                            dbc.Col(dcc.Graph(id = snapshot_30_plot_id, className="h-100") , width = {"size": 7}, style={'margin-right': '0px', 'margin-left': '0px','height': 240})
                            ]),
                    ], width={ "offset": 0}, style = {"heights":350})
                ]),

            create_store_for_graph(number),

            html.Hr(),
        ], id= display, style= {'display': 'block'})

    return both_filter_graph_layout


In [81]:
#external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__,  external_stylesheets= [dbc.themes.BOOTSTRAP])

app.layout = html.Div([

    html.H4(
        children='DTC Data Visualization',
        style={
            'textAlign': 'center'
        }
    ),

    html.Br(),

    dmc.Space(h=10),

    html.Div(children = [
        html.Label('Choose a model'),
        dmc.Space(h=10),
        dmc.Chips(
            id="model-type",
            data=[ 
                {"value":i , "label":i} for i in carmodel[carmodel['Part Number'].isin(part_numbers)]['Model'].to_list()
            ],
            value= "521H PP2",
        ),
        
    ]),

    html.Br(),

    html.Div([
        create_dtc_plot_with_summary(),

        dmc.Header(
                    height=60, 
                    children=[
                            
                   
                        dbc.Row([
                            dbc.Col([dmc.Text(
                                "Snapshot Analysis",
                                style={"fontSize": 35})]),
                            dbc.Col([dmc.Text("Filters")],  width={"size": 1, "offset":4}),
                            dbc.Col([
                                dcc.Dropdown(
                                    id ="filter-number",
                                    options = [
                                        {"value": 1, "label": "1"},
                                        {"value": 2, "label": "2"},
                                        {"value": 3, "label": "3"},
                                        {"value": 4, "label": "4"},
                                    ]
                                ),
                        ], width={"size": 1 })
                   
                    ], 
                    style={"backgroundColor": "#FFFFFF"},align="center",
                ),

            
            html.Br(),

            create_snapshot_graph_layout_1(1),

            create_snapshot_graph_layout_1(2),

            create_snapshot_graph_layout_1(3),

            create_snapshot_graph_layout_1(4),
            
           

        ]),

    ]),


],
)

  
          


@app.callback(
    Output("fyi-1", "style"),
    Output("fyi-2", "style"),
    Output("fyi-3", "style"),
    Output("fyi-4", "style"),
    Input("filter-number", "value")
)
def select_value(value):
    if value == 1:
        return {'display':'block'},{'display':'none'},{'display':'none'},{'display':'none'}
    elif value ==2:
        return {'display':'block'},{'display':'block'},{'display':'none'},{'display':'none'}
    elif value == 3:
        return {'display':'block'},{'display':'block'},{'display':'block'},{'display':'none'}
    else:
        return {'display':'block'},{'display':'block'},{'display':'block'},{'display':'block'}

app.run_server(mode='inline', debug=True, port=8020 ) 
                    

In [15]:
import dash_mantine_components as dmc
from dash import Output, Input, callback

dmc.Select(
    data=["USDINR", "EURUSD", "USDTWD", "USDJPY"],
    id="select-error",
    value="USDJPY",
    style={"width": 200},
)


@callback(Output("select-error", "error"), Input("select-error", "value"))
def select_value(value):
    return "JPY is not allowed!" if value == "USDJPY" else ""

In [5]:
dtc_list['407462']['30'].keys()

dict_keys(['FD07', 'FD09', 'FD21', 'FD23', 'FD24'])

In [6]:
def plot_snapshots_30(df,did_information, dtc_value, did_value,signal):
    snapshot_value = '30'
    try:
        track, timeseries_signal = re.split('(?<=\w\d)[_](?=[0-9A-Za-z])', signal)
    except :
        string_split = re.split('(?<=\w\d)[_](?=[0-9A-Za-z])', signal)
        track = string_split[0]
        timeseries_signal = '_'.join(string_split[1:])


    plot_signal_df =  df[df.columns[(df.columns.str.contains(rf'\d+/{dtc_value}/{snapshot_value}/{did_value}/(\w+)'))]].dropna()
    frequency_df = plot_signal_df.loc[:,plot_signal_df.columns.str.contains('Sample_rate_Ts')== True].mode()
    sample_rate = list(frequency_df.values[0])[0]/1000
    total_plotting_df = plot_signal_df.loc[:,plot_signal_df.columns.str.contains(timeseries_signal)== True]

    # if len(filter_index) <= 1 or None:
    plotting_df = total_plotting_df.copy()
    # else:
    #     plotting_df = total_plotting_df.filter(items=filter_index, axis = 0)

    if plotting_df.iloc[1,:].dtype == float or plotting_df.iloc[1,:].dtype == int:
        
        if len(np.unique(plotting_df)) > 16:
            plt_fig_30 = plot_parallel_floatdf_timeseries(plotting_df[natsorted(plotting_df.columns)],sample_rate, signal)
           
        else:
            lower_timeseries_signal= timeseries_signal.lower()
            did_signal_df = did_information[(did_information['Parameter Name'].str.contains(lower_timeseries_signal)) & (did_information['Identifier']== did_value)]
            plt_fig_30 = plot_parallel_floatdf(plotting_df[natsorted(plotting_df.columns)], did_signal_df, sample_rate, signal)
        
    else:
        plt_fig_30 = plot_parallel_stringdf(plotting_df[natsorted(plotting_df.columns)], sample_rate, signal)
       

    return plt_fig_30



In [7]:
trace_data = plot_snapshots_30(df, did_information, '407462', 'FD24','Tr24_Vehicle_speed_from_ABS_Scn_Vspd_ABS')
fig = go.Figure(trace_data)
fig.show()

C:\Users\ASRIDHAR\AppData\Local\Temp\ipykernel_36932\683864427.py:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  plot_signal_df =  df[df.columns[(df.columns.str.contains(rf'\d+/{dtc_value}/{snapshot_value}/{did_value}/(\w+)'))]].dropna()


In [29]:
dtc_value = '407462', 
snapshot_value = '30'
did_value = 'FD24'
df.columns

Index(['filename', 'vin', 'diagnostic_part_number', 'odometer',
       'readout_local_ts', '1630/0C1900/20/0330/Svt_ActAng_A',
       '1630/0C1900/20/D127/Scm_5V_Volt', '1630/0C1900/20/D904/Scn_Impact_BUS',
       '1630/0C1900/20/D94F/Yon_MainRelay',
       '1630/0C1900/20/D950/Yon_StaRelay_On',
       ...
       '1630/407462/null/05/OCC5', '1630/407462/null/06/OCC6',
       '1630/407462/null/07/OCC7', '1630/407462/null/10/FDC10',
       '1630/407462/null/12/FDC12', '1630/407462/null/20/TS20',
       '1630/407462/null/21/TS21', '1630/407462/null/30/status_indicator',
       '1630/407462/null/null/status_byte',
       '1630/null/null/null/err__decoding_failed'],
      dtype='object', length=7670)

In [39]:
df[df.columns[df.columns.str.contains(r'\d+/(\d+)/30/FD24/(\w+)')]].dropna()

C:\Users\ASRIDHAR\AppData\Local\Temp\ipykernel_104048\4257869218.py:1: UserWarning:

This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.



,1630/407462/30/FD24/Tr24_Number_of_samples_Xn,1630/407462/30/FD24/Tr24_S10_Vehicle_speed_from_ABS_Scn_Vspd_ABS,1630/407462/30/FD24/Tr24_S11_Vehicle_speed_from_ABS_Scn_Vspd_ABS,1630/407462/30/FD24/Tr24_S12_Vehicle_speed_from_ABS_Scn_Vspd_ABS,1630/407462/30/FD24/Tr24_S13_Vehicle_speed_from_ABS_Scn_Vspd_ABS,1630/407462/30/FD24/Tr24_S14_Vehicle_speed_from_ABS_Scn_Vspd_ABS,1630/407462/30/FD24/Tr24_S15_Vehicle_speed_from_ABS_Scn_Vspd_ABS,1630/407462/30/FD24/Tr24_S16_Vehicle_speed_from_ABS_Scn_Vspd_ABS,1630/407462/30/FD24/Tr24_S17_Vehicle_speed_from_ABS_Scn_Vspd_ABS,1630/407462/30/FD24/Tr24_S18_Vehicle_speed_from_ABS_Scn_Vspd_ABS,...,1630/407462/30/FD24/Tr24_S3_Vehicle_speed_from_ABS_Scn_Vspd_ABS,1630/407462/30/FD24/Tr24_S40_Vehicle_speed_from_ABS_Scn_Vspd_ABS,1630/407462/30/FD24/Tr24_S4_Vehicle_speed_from_ABS_Scn_Vspd_ABS,1630/407462/30/FD24/Tr24_S5_Vehicle_speed_from_ABS_Scn_Vspd_ABS,1630/407462/30/FD24/Tr24_S6_Vehicle_speed_from_ABS_Scn_Vspd_ABS,1630/407462/30/FD24/Tr24_S7_Vehicle_speed_from_ABS_Scn_Vspd_ABS,1630/407462/30/FD24/Tr24_S8_Vehicle_speed_from_ABS_Scn_Vspd_ABS,1630/407462/30/FD24/Tr24_S9_Vehicle_speed_from_ABS_Scn_Vspd_ABS,1630/407462/30/FD24/Tr24_Sample_rate_Ts,1630/407462/30/FD24/Tr24_Time_from_trig_up_to_end_of_last_sample_Td
0,40.0,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,...,0.10,29.98,0.10,0.10,0.10,0.10,0.10,0.10,160.0,121.0
2,40.0,0.27,0.27,0.25,0.25,0.27,0.27,0.27,0.27,0.27,...,0.27,2.79,0.27,0.27,0.25,0.25,0.27,0.27,160.0,101.0
3,40.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,160.0,121.0
5,40.0,0.00,0.00,0.17,0.62,0.63,0.63,0.37,0.27,0.27,...,0.00,2.31,0.00,0.00,0.00,0.00,0.00,0.00,160.0,142.0
6,40.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,160.0,160.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,40.0,86.05,86.41,86.82,86.99,87.31,87.57,87.74,88.00,88.27,...,83.94,87.02,84.22,84.53,84.95,85.19,85.48,85.86,160.0,100.0
699,40.0,0.11,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.11,0.00,0.11,0.11,0.11,0.11,0.11,0.11,160.0,101.0
700,40.0,0.17,0.14,0.13,0.00,0.00,0.00,0.00,0.00,0.00,...,0.42,0.00,0.37,0.34,0.27,0.24,0.21,0.20,160.0,59.0
703,40.0,0.13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.41,0.00,0.31,0.24,0.18,0.00,0.13,0.13,160.0,142.0
